In [33]:
import json


with open('/home/nermakovaa/semester_4/SNA/SNA/data/data.json', 'r') as file:
    data = json.load(file)
    
print(data)

[{'vk': [{'groupId': '19817989', 'posts': [{'id': 2221808, 'text': 'А еще мне удалось попробовать еще одну #ВВновинку -Пирожки постные с рисом и грибами. Пирожки-великолепные. Хорошее тесто и много начинки! Грибочки в начинке -очень понравились!', 'views': None, 'forwards': 0, 'replies': [{'text': 'Мне они тоже понравились 🔥', 'sender_id': 103700096, 'date': 1713129164, 'reactions': [{'emoji': 'like', 'count': 4}], 'replies': 2, 'sender_name': 'Любовь', 'last_name': 'Креминская', 'sex': 1, 'city': 'Санкт-Петербург', 'bdate': '23.4'}], 'date': 1713127426, 'reactions': [{'emoji': 'like', 'count': 30}]}, {'id': 2221793, 'text': 'Хотела поделиться впечатлением об 🔥 этом печенье-#ВВновинку Печенье сахарное с изюмом. Очень вкусное печенье,нежное на вкус. Изюм придает пикантность. Пачку очень быстро съели! Всем советуем!', 'views': None, 'forwards': 0, 'replies': [{'text': 'Мы только первую быстро скушали, ❤ а после второй решили, что наелись и больше не хотим 😂', 'sender_id': 380405, 'date':

## Tg

### Love Rate

In [5]:
def love_rate(data):
    """
    Коэффициент привлекательности (Love Rate)
    (Количество лайков / Общее количество уникальных просмотров ) * 100%
    """
    total_views = 0
    total_likes = 0

    for i in data:
        for item in i['tg']:
            total_views += sum([0 if _["views"] is None else _["views"] for _ in item["posts"]])
            total_likes += sum([reaction["count"] for post in item["posts"] for reaction in post["reactions"] if reaction["emoji"] == "❤"])

    if total_views > 0 and total_likes > 0:
        love_rate = (total_likes / total_views) * 100
        return love_rate
    else:
        return 0
    
    
love_rate(data)

1.1029618652810864

### Engagement Rate 

In [6]:
def engagement_rate(data):
    '''
    Коэффициент вовлеченности (Engagement Rate)
    (Общее число взаимодействий / Общее число подписчиков) * 100%
    '''
    total_interactions = 0
    total_comments = 0

    for i in data:
        for item in i['tg']:
            followers = item['membersCount']
            likes = sum(reaction['count'] for post in item['posts'] for reaction in post['reactions'])
            reposts = sum(post['forwards'] for post in item['posts'])

            unique_sender_ids = set(comment['sender_id'] for post in item['posts'] for comment in post.get('replies', []))

            total_comments += len(unique_sender_ids)
            total_interactions += likes + total_comments + reposts

    if total_interactions > 0 and followers > 0:
        engagement_rate = (total_interactions / followers) * 100
        return engagement_rate
    else:
        return 0


engagement_rate(data)

0.9626264357963312

### Engagement Rate By Reach

In [23]:
def engagement_rate_by_reach(data):
    '''
    Коэффициент вовлеченности (Engagement Rate By Reach)
    (Общее число взаимодействий / Общее количество уникальных просмотров) * 100%
    '''
    total_interactions = 0
    total_comments = 0
    total_views = 0

    for i in data:
        for item in i['tg']:
            total_views += sum([0 if _["views"] is None else _["views"] for _ in item["posts"]])
            
            likes = sum(reaction['count'] for post in item['posts'] for reaction in post['reactions'])
            reposts = sum(post.get('forwards', 0) for post in item['posts'])  

            unique_sender_ids = set(comment['sender_id'] for post in item['posts'] for comment in post.get('replies', []))

            total_comments += len(unique_sender_ids)
            total_interactions += likes + total_comments + reposts

    if total_interactions > 0 and total_views > 0:
        engagement_rate_by_reach = (total_interactions / total_views) * 100
        return engagement_rate_by_reach
    else:
        return 0


engagement_rate_by_reach(data)

2.106507099848061

### Показатель аудиторного охвата

In [65]:
from datetime import datetime


def audience_coverage(data):
    '''
    Показатель аудиторного охвата (Audience Coverage)
    (Количество лайков/просмотров/репостов/комментариев уникальных пользователей) / (Количество дней рассматриваемого периода) 
    '''
    for i in data:
        for item in i['tg']:
            total_interactions = item["membersCount"]
            
            start_date = datetime.strptime(item["from"], "%d/%m/%Y")
            end_date = datetime.strptime(item["to"], "%d/%m/%Y")
            days_between = (end_date - start_date).days
                
    if total_interactions > 0 and days_between > 0:
        audience_coverage = (total_interactions / days_between) 
        return int(audience_coverage)
    else:
        return 0
    
    
audience_coverage(data)

23332

### Channel Citation Index

In [77]:
def channel_citation_index(data):
    """
    Индекс цитируемости (Channel Citation Index)
    (Количество репостов / Общее количество уникальных просмотров) * 100%
    """
    total_views = 0
    total_reposts = 0

    for i in data:
        for item in i['tg']:
            valid_posts = [post for post in item["posts"] if post["views"] is not None] 
            total_views += sum([post["views"] for post in valid_posts])
            total_reposts = sum(post.get('forwards', 0) for post in item['posts'])  

    if total_views > 0 and total_reposts > 0:
        channel_citation_index = (total_reposts / total_views) * 100
        return channel_citation_index
    else:
        return 0
    
    
channel_citation_index(data)

0.25698261146855245

### Discussion Rate

In [127]:
def discussion_rate(data):
    """
    Обсуждаемость постов (Discussion Rate)
    (Количество подкомментариев) / (Количество уникальных пользователей в подкомментариях) * 100% 
    """
    for i in data:
        for item in i['tg']:
            comment = [comment['sender_id'] for post in item['posts'] for comment in post.get('replies', [])]
            
            total_comment = len(comment)
            unique_sender_ids = len(set(comment))
            
    if unique_sender_ids == 1:
        return 1
    else:
        discussion_rate = (total_comment / unique_sender_ids) * 100
        return discussion_rate
            

discussion_rate(data)

150.0

### Sentimental Analysis for messages

In [8]:
from transformers import pipeline


def top_regions(data):
    '''
    Топ-20 регионов с наибольшим числом пользователей
    [регион | пользователей | % | тональность]
    '''
    # https://huggingface.co/blanchefort/rubert-base-cased-sentiment модель для анализа тональности
    classifier = pipeline("sentiment-analysis", model="blanchefort/rubert-base-cased-sentiment") 

    city_users = {}

    total_users = 0
    
    for i in data:
        for item in i['tg']:
            for post in item['posts']:
                for reply in post.get('replies', []):
                    city = reply.get('city', 'Unknown')
                    if city not in city_users:
                        city_users[city] = []
                    city_users[city].append(reply['sender_id'])
                    total_users += 1

    city_stats = {}
    for city, users in city_users.items():
        users_count = len(users)
        percent = (users_count / total_users) * 100

        user_messages = [post['text'] for post in item['posts'] for post in post['replies'] if post.get('city') == city and post['text'] is not None]
        classified_messages = classifier(user_messages)
        len_messages = len(classified_messages)
        positive = len([i for i in classified_messages if i['label']=='POSITIVE']) / len_messages
        negative = len([i for i in classified_messages if i['label']=='NEGATIVE']) / len_messages
        neutral = len([i for i in classified_messages if i['label']=='NEUTRAL']) / len_messages

        city_stats[city] = {
            'users_count': users_count,
            'percent': percent,
            'user_messages': user_messages,
            'positive': positive,
            'negative': negative,
            'neutral': neutral
        }

    # Оставляем топ-20, сортируем city_stats по users_count 
    sorted_city_stats = dict(sorted(city_stats.items(), key=lambda x: x[1]['users_count'], reverse=True)[:20])

    top_regions = json.dumps(sorted_city_stats, indent=4, ensure_ascii=False)
    return top_regions


result = top_regions(data)
print(result)

{
    "null": {
        "users_count": 9,
        "percent": 100.0,
        "user_messages": [
            "А почему у вас нет воды легенда гор в 19 литровых бутылках?))",
            "Бенто со сгущенкой",
            "О, словечки вернулись",
            "Добрый день, Алина",
            "Полностью поддерживаем вас в этом выборе",
            "Я на хлебной планете уже 7 лет)",
            "Любимое мороженое. И на Пломбирной планете можно было бы пополнять бесконечно ведерко",
            "Промокодов для постоянных клиентов давно не было("
        ],
        "positive": 0.125,
        "negative": 0.125,
        "neutral": 0.75
    }
}


### Net Promoter Score

In [113]:
from transformers import pipeline


def net_promoter_score(data):
    """
    Лояльность пользователей (Net Promoter Score)
    ((Положительные комментарии - Негативные комментарии) / Всего комментариев) * 100%
    """
    classifier = pipeline("sentiment-analysis", model="blanchefort/rubert-base-cased-sentiment") 

    for i in data:
        for item in i['tg']:
            total_messages = [post['text'] for post in item['posts'] for post in post['replies'] if post['text'] is not None]
            print(total_messages)
            len_total_messages = len(total_messages)
            
            classified_messages = classifier(total_messages)
            positive_count = sum(1 for i in classified_messages if i['label'] == 'POSITIVE')
            negative_count = sum(1 for i in classified_messages if i['label'] == 'NEGATIVE')

    if len_total_messages > 0:
        net_promoter_score = ((positive_count - negative_count) / len_total_messages) * 100
        return net_promoter_score
    else:
        return 0
    
    
net_promoter_score(data)

['А почему у вас нет воды легенда гор в 19 литровых бутылках?))', 'Бенто со сгущенкой', 'О, словечки вернулись', 'Добрый день, Алина', 'Полностью поддерживаем вас в этом выборе', 'Я на хлебной планете уже 7 лет)', 'Любимое мороженое. И на Пломбирной планете можно было бы пополнять бесконечно ведерко', 'Промокодов для постоянных клиентов давно не было(']


0.0

### Тональность комментариев по длине символов

In [195]:
from transformers import pipeline


def character_length(data):
    '''
    Распределение тональности комментариев по длине символов
    [количество символов | тональность]
    '''
    lists = []
    results = []
    
    # https://huggingface.co/blanchefort/rubert-base-cased-sentiment модель для анализа тональности
    classifier = pipeline("sentiment-analysis", model="blanchefort/rubert-base-cased-sentiment") 
    
    # диапазоны длин символов
    dict_lengths = {1: '0-10', 2: '11-50', 3: '51-100', 4: '101-200', 5: '201+'}

    comments_lengths = {
        dict_lengths[1]: [],
        dict_lengths[2]: [],
        dict_lengths[3]: [],
        dict_lengths[4]: [],
        dict_lengths[5]: [] 
    }

    for group in data:
        for post in group['tg'][0]['posts']:
            for reply in post['replies']:
                if reply['text'] != None:
                    comment_length = len(reply['text'])
                    if comment_length < int(dict_lengths[1].split('-')[1])+1:
                        comments_lengths[dict_lengths[1]].append(reply['text'])
                    elif int(dict_lengths[1].split('-')[1])+1 <= comment_length <= int(dict_lengths[2].split('-')[1])+1:
                        comments_lengths[dict_lengths[2]].append(reply['text'])
                    elif int(dict_lengths[2].split('-')[1])+1 <= comment_length <= int(dict_lengths[3].split('-')[1])+1:
                        comments_lengths[dict_lengths[3]].append(reply['text'])
                    elif int(dict_lengths[3].split('-')[1])+1 <= comment_length <= int(dict_lengths[4].split('-')[1])+1:
                        comments_lengths[dict_lengths[4]].append(reply['text'])
                    else:
                        comments_lengths[dict_lengths[5]].append(reply['text'])

    for lengths in comments_lengths.keys():
        lists.append([classifier(_) for _ in comments_lengths[lengths]])
        
    length_keys = dict_lengths.values()

    for list in lists:
        total_score = sum(dict['score'] for sublist in list for dict in sublist)

        distribution = {'positive': 0, 'negative': 0, 'neutral': 0} 
        if total_score != 0:
            for sublist in list:
                for dict in sublist:
                    distribution[dict['label'].lower()] += (dict['score'] / total_score) * 100

        results.append(distribution)

    comments_lengths_sentimentary = {}
    for i, dist in zip(length_keys, results):
        comments_lengths_sentimentary[i] = dist

    return comments_lengths_sentimentary


character_length(data)

{'0-10': {'positive': 0, 'negative': 0, 'neutral': 0},
 '11-50': {'positive': 0,
  'negative': 19.524068842154584,
  'neutral': 80.47593115784542},
 '51-100': {'positive': 54.90797873326216,
  'negative': 0,
  'neutral': 45.092021266737845},
 '101-200': {'positive': 0, 'negative': 0, 'neutral': 0},
 '201+': {'positive': 0, 'negative': 0, 'neutral': 0}}

### Топ эмодзи

In [16]:
def top_emoji(data):
    '''
    Топ-5 эмодзи
    [эмодзи | количество упоминаний эмодзи]
    '''
    reactions_summary = {}
    
    for group in data:
        posts = (post_item for item in group['tg'] for post_item in item['posts'])
        
        for post_item in posts:
            reactions = post_item.get('reactions', {})
            
            if reactions is not None:
                for reaction in filter(lambda r: r['emoji'] is not None, reactions):
                    emoji = reaction['emoji']
                    count = reaction['count']
                    
                    reactions_summary[emoji] = reactions_summary.get(emoji, 0) + count

    top_reactions = dict(sorted(reactions_summary.items(), key=lambda x: x[1], reverse=True)[:5])
    return top_reactions  
    
    
top_emoji(data)

{'❤': 588, '👍': 82, '🔥': 74, '🤣': 71, '👎': 36}

## ВКонтакте

### Love Rate

In [19]:
def love_rate(data):
    """
    Коэффициент привлекательности (Love Rate)
    (Количество лайков / Общее количество уникальных просмотров) * 100%
    """
    total_views = 0
    total_likes = 0

    for i in data:
        for item in i['vk']:
            valid_posts = [post for post in item["posts"] if post["views"] is not None] # только те посты, где просмотры != None
            total_views += sum([post["views"] for post in valid_posts])
            total_likes += sum([reaction["count"] for post in valid_posts for reaction in post["reactions"] if reaction["emoji"] == "like"])

    if total_views > 0 and total_likes > 0:
        love_rate = (total_likes / total_views) * 100
        return love_rate
    else:
        return 0
    
    
love_rate(data)

35.0

### Engagement Rate 

In [29]:
def engagement_rate(data):
    '''
    Коэффициент вовлеченности (Engagement Rate)
    (Общее число взаимодействий / Общее число подписчиков) * 100%
    '''
    total_interactions = 0
    total_comments = 0

    for i in data:
        for item in i['vk']:
            followers = item['membersCount']
            likes = sum(reaction['count'] for post in item['posts'] for reaction in post['reactions'])
            reposts = sum(post['forwards'] for post in item['posts'])

            unique_sender_ids = set(comment['sender_id'] for post in item['posts'] for comment in post.get('replies', []))

            total_comments += len(unique_sender_ids)
            total_interactions += likes + total_comments + reposts

    if total_interactions > 0 and followers > 0:
        engagement_rate = (total_interactions / followers) * 100
        return engagement_rate
    else:
        return 0


engagement_rate(data)

0.135772935928105

### Engagement Rate By Reach

In [22]:
def engagement_rate_by_reach(data):
    '''
    Коэффициент вовлеченности (Engagement Rate By Reach)
    (Общее число взаимодействий / Общее количество уникальных просмотров) * 100%
    '''
    total_interactions = 0
    total_comments = 0
    total_views = 0

    for i in data:
        for item in i['vk']:
            valid_posts = [post for post in item["posts"] if post["views"] is not None] 
            total_views += sum([post["views"] for post in valid_posts])
            
            likes = sum(reaction['count'] for post in item['posts'] for reaction in post['reactions'])
            reposts = sum(post.get('forwards', 0) for post in item['posts'])  

            unique_sender_ids = set(comment['sender_id'] for post in item['posts'] for comment in post.get('replies', []))

            total_comments += len(unique_sender_ids)
            total_interactions += likes + total_comments + reposts

    if total_interactions > 0 and total_views > 0:
        engagement_rate_by_reach = (total_interactions / total_views) * 100
        return engagement_rate_by_reach
    else:
        return 0


engagement_rate_by_reach(data)

87.5

### Показатель аудиторного охвата

In [78]:
from datetime import datetime


def audience_coverage(data):
    '''
    Показатель аудиторного охвата (Audience Coverage)
    (Уникальные пользователи (Количество лайков + количество просмотров + количество репостов + количестко комментариев + 
    количество лайков комментариев)) / (Количество дней рассматриваемого периода) 
    '''
    total_interactions = 0
    total_comments = 0
    total_views = 0

    for i in data:
        for item in i['vk']:
            valid_posts = [post for post in item["posts"] if post["views"] is not None] 
            total_views += sum([post["views"] for post in valid_posts])
            
            likes_comments = sum([reaction['count'] for post in item['posts'] for comment in post.get('replies', []) for reaction in comment.get('reactions', [])])
            total_likes = sum(reaction['count'] for post in item['posts'] for reaction in post['reactions']) + likes_comments
            
            reposts = sum(post.get('forwards', 0) for post in item['posts'])  

            unique_sender_ids = set(comment['sender_id'] for post in item['posts'] for comment in post.get('replies', []))

            total_comments += len(unique_sender_ids)
            total_interactions += total_likes + total_comments + reposts + total_views
            
            start_date = datetime.strptime(item["from"], "%d/%m/%Y")
            end_date = datetime.strptime(item["to"], "%d/%m/%Y")
            days_between = (end_date - start_date).days
                
    if total_interactions > 0 and days_between > 0:
        audience_coverage = (total_interactions / days_between) 
        return int(audience_coverage)
    else:
        return 0
    
    
audience_coverage(data)

52

### Channel Citation Index

In [79]:
def channel_citation_index(data):
    """
    Индекс цитируемости (Channel Citation Index)
    (Количество репостов / Общее количество уникальных просмотров) * 100%
    """
    total_views = 0
    total_reposts = 0

    for i in data:
        for item in i['vk']:
            valid_posts = [post for post in item["posts"] if post["views"] is not None] 
            total_views += sum([post["views"] for post in valid_posts])
            total_reposts = sum(post.get('forwards', 0) for post in item['posts'])  

    if total_views > 0 and total_reposts > 0:
        channel_citation_index = (total_reposts / total_views) * 100
        return channel_citation_index
    else:
        return 0
    
    
channel_citation_index(data)

0

### Discussion Rate

In [128]:
def discussion_rate(data):
    """
    Обсуждаемость постов (Discussion Rate)
    (Количество подкомментариев / Количество уникальных пользователей в подкомментариях) * 100% 
    """
    for i in data:
        for item in i['vk']:
            comment = [comment['sender_id'] for post in item['posts'] for comment in post.get('replies', [])]
            
            total_comment = len(comment)
            unique_sender_ids = len(set(comment))
            
    if unique_sender_ids == 1:
        return 1
    else:
        discussion_rate = (total_comment / unique_sender_ids) * 100
        return discussion_rate
            

discussion_rate(data)

114.28571428571428

### Sentimental Analysis for messages

In [28]:
from transformers import pipeline


def top_regions(data):
    '''
    Топ-20 регионов с наибольшим числом пользователей
    [регион | пользователей | % | тональность]
    '''
    # https://huggingface.co/blanchefort/rubert-base-cased-sentiment модель для анализа тональности
    classifier = pipeline("sentiment-analysis", model="blanchefort/rubert-base-cased-sentiment") 

    city_users = {}

    total_users = 0
    
    for i in data:
        for item in i['vk']:
            for post in item['posts']:
                for reply in post.get('replies', []):
                    city = reply.get('city', 'Unknown')
                    if city not in city_users:
                        city_users[city] = []
                    city_users[city].append(reply['sender_id'])
                    total_users += 1

    city_stats = {}
    for city, users in city_users.items():
        users_count = len(users)
        percent = (users_count / total_users) * 100

        user_messages = [post['text'] for post in item['posts'] for post in post['replies'] if post.get('city') == city and post['text'] is not None]
        classified_messages = classifier(user_messages)
        len_messages = len(classified_messages)
        positive = len([i for i in classified_messages if i['label']=='POSITIVE']) / len_messages
        negative = len([i for i in classified_messages if i['label']=='NEGATIVE']) / len_messages
        neutral = len([i for i in classified_messages if i['label']=='NEUTRAL']) / len_messages

        city_stats[city] = {
            'users_count': users_count,
            'percent': percent,
            'user_messages': user_messages,
            'positive': positive,
            'negative': negative,
            'neutral': neutral
        }

    # Оставляем топ-20, сортируем city_stats по users_count 
    sorted_city_stats = dict(sorted(city_stats.items(), key=lambda x: x[1]['users_count'], reverse=True)[:20])

    top_regions = json.dumps(sorted_city_stats, indent=4, ensure_ascii=False)
    return top_regions


result = top_regions(data)
print(result)

{
    "Москва": {
        "users_count": 4,
        "percent": 50.0,
        "user_messages": [
            "Мы только первую быстро скушали, а после второй решили, что наелись и больше не хотим 😂",
            "Надо брать однозначно",
            "Пойду запускать ракету всем хорошего вечера и до новых встреч",
            "Сказочное фото, спокойной ночи"
        ],
        "positive": 0.5,
        "negative": 0.25,
        "neutral": 0.25
    },
    "Санкт-Петербург": {
        "users_count": 1,
        "percent": 12.5,
        "user_messages": [
            "Мне они тоже понравились"
        ],
        "positive": 1.0,
        "negative": 0.0,
        "neutral": 0.0
    },
    "Балашиха": {
        "users_count": 1,
        "percent": 12.5,
        "user_messages": [
            "У нас тоже быстро закончилось."
        ],
        "positive": 0.0,
        "negative": 1.0,
        "neutral": 0.0
    },
    "null": {
        "users_count": 1,
        "percent": 12.5,
        "user_messa

### Net Promoter Score

In [107]:
from transformers import pipeline


def net_promoter_score(data):
    """
    Лояльность пользователей (Net Promoter Score)
    ((Положительные комментарии - Негативные комментарии) / Всего комментариев) * 100%
    """
    # https://huggingface.co/blanchefort/rubert-base-cased-sentiment модель для анализа тональности
    classifier = pipeline("sentiment-analysis", model="blanchefort/rubert-base-cased-sentiment") 

    for i in data:
        for item in i['vk']:
            total_messages = [post['text'] for post in item['posts'] for post in post['replies'] if post['text'] is not None]
            len_total_messages = len(total_messages)
            
            classified_messages = classifier(total_messages)
            positive_count = sum(1 for i in classified_messages if i['label'] == 'POSITIVE')
            negative_count = sum(1 for i in classified_messages if i['label'] == 'NEGATIVE')

    if len_total_messages > 0:
        net_promoter_score = ((positive_count - negative_count) / len_total_messages) * 100
        return net_promoter_score
    else:
        return 0
    
    
net_promoter_score(data)

37.5

### Тональность комментариев по длине символов

In [196]:
from transformers import pipeline


def character_length(data):
    '''
    Распределение тональности комментариев по длине символов
    [количество символов | тональность]
    '''
    lists = []
    results = []
    
    # https://huggingface.co/blanchefort/rubert-base-cased-sentiment модель для анализа тональности
    classifier = pipeline("sentiment-analysis", model="blanchefort/rubert-base-cased-sentiment") 
    
    # диапазоны длин символов
    dict_lengths = {1: '0-10', 2: '11-50', 3: '51-100', 4: '101-200', 5: '201+'}

    comments_lengths = {
        dict_lengths[1]: [],
        dict_lengths[2]: [],
        dict_lengths[3]: [],
        dict_lengths[4]: [],
        dict_lengths[5]: [] 
    }

    for group in data:
        for post in group['vk'][0]['posts']:
            for reply in post['replies']:
                if reply['text'] != None:
                    comment_length = len(reply['text'])
                    if comment_length < int(dict_lengths[1].split('-')[1])+1:
                        comments_lengths[dict_lengths[1]].append(reply['text'])
                    elif int(dict_lengths[1].split('-')[1])+1 <= comment_length <= int(dict_lengths[2].split('-')[1])+1:
                        comments_lengths[dict_lengths[2]].append(reply['text'])
                    elif int(dict_lengths[2].split('-')[1])+1 <= comment_length <= int(dict_lengths[3].split('-')[1])+1:
                        comments_lengths[dict_lengths[3]].append(reply['text'])
                    elif int(dict_lengths[3].split('-')[1])+1 <= comment_length <= int(dict_lengths[4].split('-')[1])+1:
                        comments_lengths[dict_lengths[4]].append(reply['text'])
                    else:
                        comments_lengths[dict_lengths[5]].append(reply['text'])

    for lengths in comments_lengths.keys():
        lists.append([classifier(_) for _ in comments_lengths[lengths]])
        
    length_keys = dict_lengths.values()

    for list in lists:
        total_score = sum(dict['score'] for sublist in list for dict in sublist)

        distribution = {'positive': 0, 'negative': 0, 'neutral': 0} 
        if total_score != 0:
            for sublist in list:
                for dict in sublist:
                    distribution[dict['label'].lower()] += (dict['score'] / total_score) * 100

        results.append(distribution)

    comments_lengths_sentimentary = {}
    for i, dist in zip(length_keys, results):
        comments_lengths_sentimentary[i] = dist

    return comments_lengths_sentimentary


character_length(data)

{'0-10': {'positive': 0, 'negative': 0, 'neutral': 0},
 '11-50': {'positive': 54.046635553825936,
  'negative': 21.95435574663157,
  'neutral': 23.9990086995425},
 '51-100': {'positive': 78.39615016850013,
  'negative': 21.60384983149988,
  'neutral': 0},
 '101-200': {'positive': 0, 'negative': 0, 'neutral': 0},
 '201+': {'positive': 0, 'negative': 0, 'neutral': 0}}

### Топ эмодзи

In [31]:
import emoji
from collections import Counter


def top_emoji(data):
    '''
    Топ-5 эмодзи
    [эмодзи | количество упоминаний эмодзи в комментариях]
    '''
    def extract_emojis(text):
        return ''.join(c for c in text if emoji.is_emoji(c))

    emoji_counter = Counter()

    for group in data:
        for item in group['vk']:

            messages = (reply['text'] for post in item['posts'] 
                        for reply in post['replies'] if reply['text'] is not None)
            
            for message in messages:
                emojis = extract_emojis(message)
                emoji_counter.update(emojis)

    top_emojis = dict(emoji_counter.most_common(5))       
    return top_emojis  


top_emoji(data)

{'👎': 2, '🔥': 1, '❤': 1, '😂': 1, '⚡': 1}

## Общие метрики

### PageRank

In [568]:
import json

with open('/home/nermakovaa/semester_4/SNA/SNA/data/new_data.txt', 'r') as file:
    file_content = file.read()
    data = json.loads(file_content)

In [569]:
import networkx as nx
import matplotlib.pyplot as plt
import os


class PageRank:
    '''
    Топ-10 авторов с наибольшей способностью вовлекать других пользователей в дискуссию
    [пользователь | значение page rank]
    '''
    def __init__(self, data, top_authors=10):
        self.data = data
        self.top_authors = top_authors
        self.sender_id, self.sender_info, self.connections = self.__get_data()
        self.__get_file()
        
    def __get_data(self):
        '''
        sender_id - узлы (id пользователей)
        sender_info - (словарь с id в качестве ключа и значений в качестве словаря из first_name и last_name)
        connections - ребра (id-to-id)
        '''
        connections = []

        for post in self.data['vk'][0]['posts']:
            if post['from'] != None:
                post_connections = [(post['from']['id'], reply['sender_id']) for reply in post['replies']]
                connections.extend(post_connections)

        sender_id = list(set(sum(connections, ())))

        sender_info = {}

        for post in self.data['vk'][0]['posts']:
            if post['from'] is not None and post['from']['id'] in sender_id:
                if post['from']['id'] not in sender_info:
                    sender_info[post['from']['id']] = {
                        'first_name': post['from']['first_name'],
                        'last_name': post['from']['last_name']
                    }

        for post in self.data['vk'][0]['posts']:
            if 'replies' in post:
                for reply in post['replies']:
                    if reply['sender_id'] in sender_id:
                        if reply['sender_id'] not in sender_info:
                            sender_info[reply['sender_id']] = {
                                'first_name': reply['sender_name'],
                                'last_name': reply['last_name']
                            }

        return sender_id, sender_info, connections

    def page_rank(self):
        '''
        Для получения топа пользователей pagerank {722219350: 0.04746, 732871646: 0.03822 ...}
        '''
        G = nx.DiGraph()
        [G.add_node(k, first_name = self.sender_info[k]['first_name'], last_name = self.sender_info[k]['last_name']) for k in self.sender_id]
        G.add_edges_from(self.connections)
        pos = nx.spring_layout(G, k=0.15, iterations=20) 
        
        # pagerank - https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html
        pr = nx.pagerank(G)
        sorted_pr = sorted(pr.items(), key=lambda x: x[1], reverse=True)

        top_pr = sorted_pr[:self.top_authors] 
        
        # топ 10 пользователей с наибольшим pagerank
        name_surname_dict = {k: round(v, 5) for k, v in top_pr}
        
        return name_surname_dict # возвращает id и значение pagerank {722219350: 0.04746, 732871646: 0.03822 ...}
    
    def __get_file(self):
        '''
        Для сохранения файла pagerank
        '''
        # spring_layout - https://networkx.org/documentation/stable/reference/generated/networkx.drawing.layout.spring_layout.html
        G = nx.DiGraph()
        [G.add_node(k, first_name=self.sender_info[k]['first_name'], last_name=self.sender_info[k]['last_name']) for k in self.sender_id]
        G.add_edges_from(self.connections)
        pos = nx.spring_layout(G, k=0.15, iterations=20)

        pr = nx.pagerank(G)
        sorted_pr = sorted(pr.items(), key=lambda x: x[1], reverse=True)
        top_pr = sorted_pr[:self.top_authors]
        top_ids = [id for id, pr in top_pr]
        top_nodes = {node: G.nodes[node]['first_name'] + ' ' + G.nodes[node]['last_name'] for node in dict(top_pr).keys()}

        node_colors = ['#FF5558' if node in top_ids else "#27BBBD" for node in G.nodes()]
        plt.figure(figsize=(10, 10))
        nx.draw_networkx_labels(G, pos,
                                labels=top_nodes,
                                font_color='#333335',
                                font_size=10,
                                bbox=dict(facecolor='white', edgecolor='white', boxstyle='square'))
        nx.draw(G, pos,
                nodelist=list(pr.keys()),
                node_size=[v * 20000 for v in pr.values()],
                with_labels=False,
                node_color=node_colors,
                edge_color='#27BBBD',
                width=0.2)
        
        current_dir = os.getcwd()
        file_path = os.path.join(current_dir, 'data/PageRank.png')
        plt.savefig(file_path) 
        plt.close()  
        
    def get_table(self):
        '''
        Данные для таблицы топ-инфлюенсеров PageRank
        '''
        result_dict = {}
        for k, v in self.page_rank().items():
            name = self.sender_info[k]['first_name'] + ' ' + self.sender_info[k]['last_name']
            result_dict[name] = {
                'page_rank': self.page_rank()[k],
                'id': k
            }

        for key, value in result_dict.items():
            id_to_find = value['id']
            count = 0
            for tpl in self.connections:
                if id_to_find in tpl:
                    count += 1
            value['engagement_users'] = count
        
        for key, value in result_dict.items():
            value['comments'] = []  
            
        id = [value['id'] for key, value in result_dict.items()]

        for post in self.data['vk'][0]['posts']:
            for replies in post['replies']:
                for key, value in result_dict.items():
                    if replies['sender_id'] == value['id']:
                        value['comments'].append(replies['text'])
                        
        classifier = pipeline("sentiment-analysis", model="blanchefort/rubert-base-cased-sentiment")
        
        for key, value in result_dict.items():
            pos_count = 0
            neg_count = 0
            comments = value['comments']

            for comment in comments:
                sentiment = classifier(comment)
                if sentiment[0]['label'] == 'POSITIVE':
                    pos_count += 1
                else:
                    neg_count += 1

            if (pos_count + neg_count) != 0:
                nps = (pos_count / (pos_count + neg_count)) * 100 # nps = лояльность
                value['net_promoter_score'] = nps
            else:
                0     
                                    
        for key in result_dict:
            if 'comments' in result_dict[key]:
                del result_dict[key]['comments']
        
        return result_dict # {'Марина Вкусвилл': {'page_rank': 0.04746,  'id': 722219350,  'engagement_users': 28,  'net_promoter_score': 82.14285714285714},   
        

table = PageRank(data)
table.get_table()

{'Марина Вкусвилл': {'page_rank': 0.04746,
  'id': 722219350,
  'engagement_users': 28,
  'net_promoter_score': 82.14285714285714},
 'Дмитрий Вкусвилл': {'page_rank': 0.03822,
  'id': 732871646,
  'engagement_users': 24,
  'net_promoter_score': 41.66666666666667},
 'Регина Вкусвилл': {'page_rank': 0.03635,
  'id': 449228789,
  'engagement_users': 22,
  'net_promoter_score': 77.27272727272727},
 'Светлана Солнышко': {'page_rank': 0.03604,
  'id': 292391,
  'engagement_users': 1,
  'net_promoter_score': 0.0},
 'Виола Вкусвилл': {'page_rank': 0.03297,
  'id': 754226736,
  'engagement_users': 22,
  'net_promoter_score': 59.09090909090909},
 'Татьяна Балакирева': {'page_rank': 0.02898,
  'id': 588079193,
  'engagement_users': 81,
  'net_promoter_score': 73.33333333333333},
 'Катя Недорезова': {'page_rank': 0.02807,
  'id': 65412446,
  'engagement_users': 37,
  'net_promoter_score': 60.0},
 'Ольга Гадкова-Князева': {'page_rank': 0.02313,
  'id': 116534950,
  'engagement_users': 46,
  'net_pr

### Betweenness Centrality

In [587]:
import networkx as nx
import matplotlib.pyplot as plt
import os


class BetweennessCentralityRank:
    '''
    Топ-10 авторов с наибольшей способностью вовлекать других пользователей в дискуссию
    [пользователь | значение page rank]
    '''
    def __init__(self, data, top_authors=10):
        self.data = data
        self.top_authors = top_authors
        self.sender_id, self.sender_info, self.connections = self.__get_data()
        self.__get_file()
        
    def __get_data(self):
        '''
        sender_id - узлы (id пользователей)
        sender_info - (словарь с id в качестве ключа и значений в качестве словаря из first_name и last_name)
        connections - ребра (id-to-id)
        '''
        connections = []

        for post in self.data['vk'][0]['posts']:
            if post['from'] != None:
                post_connections = [(post['from']['id'], reply['sender_id']) for reply in post['replies']]
                connections.extend(post_connections)

        sender_id = list(set(sum(connections, ())))

        sender_info = {}

        for post in self.data['vk'][0]['posts']:
            if post['from'] is not None and post['from']['id'] in sender_id:
                if post['from']['id'] not in sender_info:
                    sender_info[post['from']['id']] = {
                        'first_name': post['from']['first_name'],
                        'last_name': post['from']['last_name']
                    }

        for post in self.data['vk'][0]['posts']:
            if 'replies' in post:
                for reply in post['replies']:
                    if reply['sender_id'] in sender_id:
                        if reply['sender_id'] not in sender_info:
                            sender_info[reply['sender_id']] = {
                                'first_name': reply['sender_name'],
                                'last_name': reply['last_name']
                            }

        return sender_id, sender_info, connections

    def page_rank(self):
        '''
        Для получения топа пользователей betweenness_centrality_rank {722219350: 0.04746, 732871646: 0.03822 ...}
        '''
        G = nx.DiGraph()
        [G.add_node(k, first_name = self.sender_info[k]['first_name'], last_name = self.sender_info[k]['last_name']) for k in self.sender_id]
        G.add_edges_from(self.connections)
        pos = nx.spring_layout(G, k=0.15, iterations=20) 
        
        # pagerank - https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html
        centrality = nx.betweenness_centrality(G)
        sorted_centrality = sorted(centrality.items(), key=lambda x: x[1], reverse=True)

        top_centrality = sorted_centrality[:self.top_authors] 
        
        # топ 10 пользователей с наибольшим pagerank
        name_surname_dict = {k: round(v, 5) for k, v in top_centrality}
        
        return name_surname_dict # возвращает id и значение pagerank {722219350: 0.04746, 732871646: 0.03822 ...}    
    
    def __get_file(self):
        '''
        Для сохранения файла pagerank
        '''
        # spring_layout - https://networkx.org/documentation/stable/reference/generated/networkx.drawing.layout.spring_layout.html
        G = nx.DiGraph()
        [G.add_node(k, first_name=self.sender_info[k]['first_name'], last_name=self.sender_info[k]['last_name']) for k in self.sender_id]
        G.add_edges_from(self.connections)
        pos = nx.spring_layout(G, k=0.15, iterations=20)

        centrality = nx.betweenness_centrality(G)
        sorted_centrality = sorted(centrality.items(), key=lambda x: x[1], reverse=True)
        top_centrality = sorted_centrality[:self.top_authors] 
        top_ids = [id for id, pr in top_centrality]
        top_nodes = {node: G.nodes[node]['first_name'] + ' ' + G.nodes[node]['last_name'] for node in dict(top_centrality).keys()}

        node_colors = ['#FF5558' if node in top_ids else '#27BBBD' for node in G.nodes()]
        plt.figure(figsize=(15, 15))
        nx.draw_networkx_labels(G, pos,
                                labels=top_nodes,
                                font_color='#333335',
                                font_size=10,
                                bbox=dict(facecolor='white', edgecolor='white', boxstyle='square'))
        nx.draw(G, pos,
                nodelist=list(centrality.keys()),
                node_size=[v * 20000 for v in centrality.values()],
                with_labels=False,
                node_color=node_colors,
                edge_color='#27BBBD',
                width=0.2)
        
        current_dir = os.getcwd()
        file_path = os.path.join(current_dir, 'data/BetweennessCentralityRank.png')
        plt.savefig(file_path) 
        plt.close()      
    
    def get_table(self):
        '''
        Данные для таблицы топ-инфлюенсеров BetweennessCentralityRank
        '''
        result_dict = {}
        for k, v in self.page_rank().items():
            name = self.sender_info[k]['first_name'] + ' ' + self.sender_info[k]['last_name']
            result_dict[name] = {
                'page_rank': self.page_rank()[k],
                'id': k
            }

        for key, value in result_dict.items():
            id_to_find = value['id']
            count = 0
            for tpl in self.connections:
                if id_to_find in tpl:
                    count += 1
            value['engagement_users'] = count
        
        for key, value in result_dict.items():
            value['comments'] = []  

        for post in self.data['vk'][0]['posts']:
            for replies in post['replies']:
                for key, value in result_dict.items():
                    if replies['sender_id'] == value['id']:
                        value['comments'].append(replies['text'])
                        
        classifier = pipeline("sentiment-analysis", model="blanchefort/rubert-base-cased-sentiment")
        
        for key, value in result_dict.items():
            comments = value['comments']
            
            pos_count = 0
            neg_count = 0
            neu_count = 0

            for comment in comments:
                sentiment = classifier(comment)
                label = sentiment[0]['label']

                if label == 'POSITIVE':
                    pos_count += 1
                elif label == 'NEGATIVE':
                    neg_count += 1
                else:
                    neu_count += 1
            
            total_count = pos_count + neg_count + neu_count
            value['positive'] = pos_count / total_count * 100
            value['negative'] = neg_count / total_count * 100
            value['neutral'] = neu_count / total_count * 100
            
        for key in result_dict:
            if 'comments' in result_dict[key]:
                del result_dict[key]['comments']
        
        return result_dict # {'Татьяна Балакирева': {'page_rank': 0.07433, 'id': 588079193,  'engagement_users': 81,  'positive': 73.33333333333333,  'negative': 8.88888888888889,  'neutral': 17.77777777777778},
    
        
table = BetweennessCentralityRank(data)
table.get_table()

{'Татьяна Балакирева': {'page_rank': 0.07433,
  'id': 588079193,
  'engagement_users': 81,
  'positive': 73.33333333333333,
  'negative': 8.88888888888889,
  'neutral': 17.77777777777778},
 'Ольга Гадкова-Князева': {'page_rank': 0.0409,
  'id': 116534950,
  'engagement_users': 46,
  'positive': 66.66666666666666,
  'negative': 29.166666666666668,
  'neutral': 4.166666666666666},
 'Юлия Корниенко': {'page_rank': 0.02592,
  'id': 11221120,
  'engagement_users': 12,
  'positive': 61.53846153846154,
  'negative': 15.384615384615385,
  'neutral': 23.076923076923077},
 'Евгения Санникова': {'page_rank': 0.02282,
  'id': 2057854,
  'engagement_users': 29,
  'positive': 41.66666666666667,
  'negative': 16.666666666666664,
  'neutral': 41.66666666666667},
 'Галина Погорельченко': {'page_rank': 0.02226,
  'id': 151635555,
  'engagement_users': 52,
  'positive': 50.0,
  'negative': 12.5,
  'neutral': 37.5},
 'Вера Михайловна': {'page_rank': 0.01697,
  'id': 635311641,
  'engagement_users': 27,
  

### Топ-5 авторов с наибольшим числом сообщений

In [178]:
import json

with open('/home/nermakovaa/semester_4/SNA/SNA/data/new_data.txt', 'r') as file:
    file_content = file.read()
    data = json.loads(file_content)

In [211]:
from transformers import pipeline

class InfluencerTable:
    '''
    Топ-5 авторов с наибольшим числом сообщений
    [пользователь | постов в группе | вовлеченность | лояльность]
    '''
    def __init__(self, data, person=5):
        self.data = data
        self.person = person
        
    def __get_data(self):
        dt = {}  

        for post in self.data['vk'][0]['posts']:
            if post['from'] is not None:
                post_id = post['from']['id']
                if post_id not in dt:
                    dt[post_id] = {
                        'first_name': post['from']['first_name'],
                        'last_name': post['from']['last_name'],
                        'likes': post['reactions'][0]['count'],
                        'messages': 1,
                        'comments': [reply['text'] for reply in post['replies']],
                        'reposts': post['forwards'],
                        'followers': self.data['vk'][0]['membersCount']
                    }
                else:  
                    dt[post_id]['messages'] += 1
                    dt[post_id]['likes'] += post['reactions'][0]['count']
                    dt[post_id]['reposts'] += post['forwards']
                    dt[post_id]['comments'].extend([reply['text'] for reply in post['replies']])
                
        return dt
            
    def get_name(self):
        '''
        ФИО пользователей с наибольшим числом инициированных постов
        '''
        self.data = self.__get_data()
        sorted_data = sorted(self.data.values(), key=lambda x: x['messages'], reverse=True)
        authors_names = {v['first_name'] + ' ' + v['last_name']: v['messages'] for v in sorted_data[:self.person]}
        return authors_names # словарь {'first_name last_name': messages}, где messages - количество постов, инициированных данным пользователем
    
    def engagement_rate_by_reach(self):
        '''
        Коэффициент вовлеченности (Engagement Rate By Reach)
        (Общее число взаимодействий / Общее количество подписчиков сообщества) * 100%
        '''
        sorted_data = sorted(self.data.values(), key=lambda x: x['messages'], reverse=True)

        engagement_rates = {
            v['first_name'] + ' ' + v['last_name']: round(((v['likes'] + v['reposts'] + len(v['comments'])) / v['followers']) * 1000, 3) 
            for v in sorted_data[:self.person] if v['followers'] != 0
        }
        return engagement_rates # словарь {'first_name last_name': engagement_rate_by_reach}, engagement_rate_by_reach - вовлеченность
    
    def net_promoter_score(self):
        """
        Лояльность пользователей (Net Promoter Score)
        ((Положительные комментарии - Негативные комментарии) / Всего комментариев) * 100%
        """
        classifier = pipeline("sentiment-analysis", model="blanchefort/rubert-base-cased-sentiment") 
        
        top_users = sorted(self.data.values(), key=lambda x: x['messages'], reverse=True)[:self.person]
        loyalty_scores = []
        
        for user in top_users:
            positive_count = 0
            negative_count = 0
            
            for comment in user['comments']:
                classified_comment = classifier(comment)
                if classified_comment[0]['label'] == 'POSITIVE':
                    positive_count += 1
                elif classified_comment[0]['label'] == 'NEGATIVE':
                    negative_count += 1
            
            total_comments = len(user['comments'])
            if total_comments > 0:
                loyalty_score = ((positive_count - negative_count) / total_comments) * 100
                loyalty_scores.append((user['first_name'] + ' ' + user['last_name'], round(loyalty_score, 2)))
        
        return sorted(loyalty_scores, key=lambda x: x[1], reverse=True) # словарь {'first_name last_name': net_promoter_score}, net_promoter_score - лояльность

    def influencer_table_result(self):
        '''
        Топ-5 авторов с наибольшим числом сообщений
        '''
        data_dict = {}
        
        names = self.get_name()
        engagement_rate = self.engagement_rate_by_reach()
        nps = self.net_promoter_score()
        
        nps_dict = {name: rating for name, rating in nps}
        
        for name, messages in names.items():
            data_dict[name] = {
                'get_message': messages,
                'engagement_rate_by_reach': engagement_rate.get(name, 0),
                'net_promoter_score': nps_dict.get(name, 0)
            }
        
        return data_dict # {'Галина Погорельченко': {'get_message': 10,  'engagement_rate_by_reach': 4.31,  'net_promoter_score': 65.0}, 'Евгения Санникова': ...}
    
us = InfluencerTable(data)
us.influencer_table_result()

{'Галина Погорельченко': {'get_message': 10,
  'engagement_rate_by_reach': 4.31,
  'net_promoter_score': 65.0},
 'Евгения Санникова': {'get_message': 7,
  'engagement_rate_by_reach': 2.635,
  'net_promoter_score': 68.42},
 'Татьяна Балакирева': {'get_message': 6,
  'engagement_rate_by_reach': 3.108,
  'net_promoter_score': 65.91},
 'Ольга Гадкова-Князева': {'get_message': 5,
  'engagement_rate_by_reach': 2.162,
  'net_promoter_score': 68.18},
 'Татьяна Алексеева': {'get_message': 4,
  'engagement_rate_by_reach': 1.727,
  'net_promoter_score': 44.44}}

### Топ постов с высоким уровнем негативной реакции аудитории

In [244]:
import json

with open('/home/nermakovaa/semester_4/SNA/SNA/data/new_data.txt', 'r') as file:
    file_content = file.read()
    data = json.loads(file_content)

In [275]:
from transformers import pipeline

class InfluencerTableNegative:
    '''
    Топ-10 постов с высоким уровнем негативной реакции аудитории
    [пост | инфлюенсер | негатив | нейтрально | позитив | вовлеченность]
    '''
    def __init__(self, data, top_neg_posts=10):
        self.data = data
        self.top_neg_posts = top_neg_posts
        self.__analyze_comments()
        self.__engagement_rate_by_reach()

    def __get_data(self):
        dt = {}  

        for post in self.data['vk'][0]['posts']:
            if post['from'] is not None:
                post_id = post['id']
                if post_id not in dt:
                    dt[post_id] = {
                        'first_name': post['from']['first_name'],
                        'last_name': post['from']['last_name'],
                        'likes': post['reactions'][0]['count'],
                        'messages': 1,
                        'comments': [reply['text'] for reply in post['replies']],
                        'reposts': post['forwards'],
                        'followers': self.data['vk'][0]['membersCount'],
                        'post': post['text']
                    }
                    
        return dt

    def __analyze_comments(self):
        self.data = self.__get_data()
        
        classifier = pipeline("sentiment-analysis", model="blanchefort/rubert-base-cased-sentiment")
        
        for id, info in self.data.items():
            pos_count = 0
            neg_count = 0
            neu_count = 0
            total_count = 0
            
            for comment in info['comments']:
                result = classifier(comment)
                sentiment = result[0]['label']
                
                if sentiment == 'NEGATIVE':
                    neg_count += 1
                elif sentiment == 'POSITIVE':
                    pos_count += 1
                else:
                    neu_count += 1
                
                total_count += 1

            pos_perc = (pos_count / total_count) * 100
            neg_perc = (neg_count / total_count) * 100
            neu_perc = (neu_count / total_count) * 100

            info['positive_percentage'] = pos_perc
            info['negative_percentage'] = neg_perc
            info['neutral_percentage'] = neu_perc

        sorted_data = sorted(self.data.items(), key=lambda x: x[1]['negative_percentage'], reverse=True)
        self.neg_posts = sorted_data[:self.top_neg_posts]
    
    def __engagement_rate_by_reach(self):
        '''
        Коэффициент вовлеченности (Engagement Rate By Reach)
        (Общее число взаимодействий / Общее количество подписчиков сообщества) * 100%
        '''
        for id, info in self.neg_posts:
            total_interactions = info['likes'] + len(info['comments']) + info['reposts']
            engagement_rate = (total_interactions / info['followers']) * 100
            info['engagement_rate'] = engagement_rate
    
    def get_results_dict(self):
        
        results_dict = {}
        
        for id, info in self.neg_posts:
            results_dict[info['post']] = {
                'first_name': info['first_name'] + ' ' + info['last_name'],
                'negative': info['negative_percentage'],
                'neutral': info['neutral_percentage'],
                'positive': info['positive_percentage'],
                'engagement_rate': info['engagement_rate']
            }
        
        # # {'Друзья, будьте осторожны...',  {'first_name': 'Наталия Милано', 'negative': 100.0,  
        # 'neutral': 0.0,  'positive': 0.0,  'engagement_rate': 0.01790601898038012}, 'Вот такие стрипсы, одни обрезки': {'first...   
        return results_dict 
    
    
table = InfluencerTableNegative(data)
table.get_results_dict()

{'Друзья, будьте осторожны с покупкой куриных крылышек во ВкусВилл!\n\nПокупала не по уценке, дата свежая из всех возможных. Вчера привезли с запахом хлора или чего-то подобного, а сегодня ранним утром крылышки уже были тухлые (хотя они ещё в рамках обещанного срока годности).\n\n[club19817989|ВкусВилл. Здесь полезное вкусно] , следите за качеством, пожалуйста. А также следите за работой службы поддержки, полная некомпетентность!!! \nЗаказ - №302666617-1\nПрошу оформить возврат если возможно, не хочется тратить время на поход в магазин. \nКрылышки в помойке где им и место.\n\n🙄upd В итоге вернула тухлые крылышки в магазин так как в онлайн возврате отказали. \nСервис и продукты фантастические 🤢🤮': {'first_name': 'Наталия Милано',
  'negative': 100.0,
  'neutral': 0.0,
  'positive': 0.0,
  'engagement_rate': 0.01790601898038012},
 'Вот такие стрипсы, одни обрезки': {'first_name': 'Людмила Большакова',
  'negative': 100.0,
  'neutral': 0.0,
  'positive': 0.0,
  'engagement_rate': 0.020464

## Модель расчета рейтинга веб-пространства компании

In [1]:
import json

with open('/home/nermakovaa/semester_4/SNA/SNA/data/new_data.txt', 'r') as file:
    file_content = file.read()
    data = json.loads(file_content)

In [132]:
from collections import defaultdict
from transformers import pipeline
import networkx as nx


class Model:
    def __init__(self, data):
        self.data = data
    
    def brand_follower_ratio(self):
        '''
        Доля подписчиков сообщества среди активных пользователей
        '''
        active_users = 0
        followers = 0

        for post in self.data['vk'][0]['posts']:
            if post['from'] != None:
                active_users += 1
                followers += post['from']['is_member']
                
        if active_users != 0:
            return followers / active_users
        else:
            return 0
    
    def user_engagement_ratio(self):
        '''
        Доля комментирующих среди активных пользователей
        '''
        post_data = defaultdict(int)
        post_count = len(self.data['vk'][0]['posts'])

        ratios = []
        for post in self.data['vk'][0]['posts']:
            sender_ids = set(comment['sender_id'] for comment in post['replies'])

            unique_sender_count = len(sender_ids)
            unique_forward_count = post['forwards']
            unique_reaction_count = post['reactions'][0]['count']

            post_data[post['id']] = {
                'unique_sender_count': unique_sender_count,
                'unique_forward_count': unique_forward_count,
                'unique_reaction_count': unique_reaction_count
            }

            ratios.append(unique_sender_count / (unique_sender_count + unique_forward_count + unique_reaction_count))

        if post_count != 0:
            return sum(ratios) / post_count
        else:
            return 0
    
    def top_audience_ratio(self):
        '''
        Доля подписчиков из топа активных пользователей
        '''
        active_users = 0
        followers = 0

        for post in self.data['vk'][0]['posts']:
            if post['from'] != None:
                sorted_posts = sorted(self.data['vk'][0]['posts'], key=lambda x: (x['views'] if x['views'] is not None else len(x['replies'])) if x['from'] is not None else -1, reverse=True)
                top_posts = sorted_posts[:10]

        for post in top_posts:
            active_users += 1
            followers += post['from']['is_member']
                
        if active_users != 0:
            return followers / active_users
        else:
            return 0
    
    def love_rate(self):
        '''
        Коэффициент привлекательности
        '''
        love_rate_sum = 0
        post_count = 0

        for post in self.data['vk'][0]['posts']:
            if post['views'] is not None:
                views = post['views']
                likes = post['reactions'][0]['count']
                
                love_rate = likes / views
                
                love_rate_sum += love_rate
                post_count += 1

        if post_count != 0:
            return (love_rate_sum / post_count) / 0.05
        else:
            return 0
            
    def trending_content_sentiment_ratio(self):
        '''
        Доля позитивных комментариев из топа наиболее обсуждаемых постов
        '''
        posts_with_replies = []

        for post in self.data['vk'][0]['posts']:
            post_id = post['id']
            replies = post['replies']
            
            if replies:
                posts_with_replies.append({'post_id': post_id, 'replies_count': len(replies), 'replies': replies})
                
        sorted_posts = sorted(posts_with_replies, key=lambda x: x['replies_count'], reverse=True)
        top_posts = sorted_posts[:10]
        
        classifier = pipeline("sentiment-analysis", model="blanchefort/rubert-base-cased-sentiment")
        post_sentiments = {}

        for post in top_posts:
            post_id = post['post_id']
            replies = post['replies']
            
            positive_count = 0
            negative_count = 0
            
            for reply in replies:
                text = reply['text']
                tone = classifier(text)[0]['label']
                
                if tone == 'POSITIVE':
                    positive_count += 1
                elif tone == 'NEGATIVE':
                    negative_count += 1
            
            post_sentiments[post_id] = {'positive_count': positive_count, 'negative_count': negative_count}

        total_loyalty = 0
        
        for post_id, counts in post_sentiments.items():
            total_comments = counts['positive_count'] + counts['negative_count']
            positive_ratio = counts['positive_count'] / total_comments if total_comments != 0 else 0

            total_loyalty += positive_ratio

        if len(post_sentiments) != 0:
            return total_loyalty / len(post_sentiments)
        else:
            return 0
            
    def net_promoter_score(self):
        '''
        Лояльность аудитории бренда
        '''
        classifier = pipeline("sentiment-analysis", model="blanchefort/rubert-base-cased-sentiment")
        post_sentiments = {}

        for post in self.data['vk'][0]['posts']:
            if len(post['replies']) != 0:
                post_id = post['id']
                replies = post['replies']
                
                positive_count = 0
                negative_count = 0
                
                for reply in replies:
                    text = reply['text']
                    tone = classifier(text)[0]['label'] 
                    
                    if tone == 'POSITIVE':
                        positive_count += 1
                    elif tone == 'NEGATIVE':
                        negative_count += 1
                
                post_sentiments[post_id] = {'positive_count': positive_count, 'negative_count': negative_count}

        total_loyalty = 0

        for post_id, counts in post_sentiments.items():
            total_comments = counts['positive_count'] + counts['negative_count']
            positive_ratio = counts['positive_count'] / total_comments if total_comments != 0 else 0

            total_loyalty += positive_ratio

        if len(post_sentiments) != 0:
            return total_loyalty / len(post_sentiments)
        else:
            return 0
        
    def brand_response_to_comments(self):
        '''
        Среднее время отклика компании на комментарии
        '''
        count_1 = 0
        count_0 = 0

        for post in self.data['vk'][0]['posts']:
            text = post['text'].lower()
            brand_name = self.data['vk'][0]['groupName'].lower()
            
            if brand_name in text:
                for reply in post['replies']:
                    if reply.get('sender_name', '').lower() in brand_name or reply.get('last_name', '').lower() in brand_name:
                        count_1 += 1
                        break
                else:
                    count_0 += 1

        total_posts = count_1 + count_0

        if total_posts != 0:
            return (count_1 + count_0) / total_posts
        else:
            return 0
        
    def brand_reaction_to_negativity(self):
        count_1 = 0
        count_0 = 0

        classifier = pipeline("sentiment-analysis", model="blanchefort/rubert-base-cased-sentiment")

        for post in self.data['vk'][0]['posts']:
            text = post['text'].lower()
            brand_name = self.data['vk'][0]['groupName'].lower()
            
            if brand_name in text:
                result = classifier(text)
                tone = result[0]['label']

                if tone == 'NEGATIVE':
                    for reply in post['replies']:
                        if reply.get('sender_name', '').lower() in brand_name or reply.get('last_name', '').lower() in brand_name:
                            count_1 += 1
                            break
                    else:
                        count_0 += 1

        total_posts = count_1 + count_0

        if total_posts != 0:
            return (count_1 + count_0) / total_posts
        else:
            return 0
        
    def brand_responsiveness(self):
        '''
        Отзывчивость бренда
        '''
        answers = 0
        posts = 0

        for post in self.data['vk'][0]['posts']:
            posts += 1
            post_answered = False  
            for replies in post['replies']:
                if (replies['sender_name'].lower() in self.data['vk'][0]['groupName'].lower()) == True or (replies['last_name'].lower() in self.data['vk'][0]['groupName'].lower()) == True:
                    answers += 1
                    post_answered = True 
                    break  
            if post_answered:  
                continue

        if posts != 0:
            return answers / posts
        else:
            return 0
        
    def influencer_sentiment_ratio(self):
        '''
        Доля лояльных инфлюенсеров из топа
        '''
        connections = []

        for post in self.data['vk'][0]['posts']:
            if post['from'] != None:
                post_connections = [(post['from']['id'], reply['sender_id']) for reply in post['replies']]
                connections.extend(post_connections)

        G = nx.DiGraph()
        G.add_edges_from(connections)
        pos = nx.spring_layout(G, k=0.15, iterations=20) 

        # pagerank - https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html
        pr = nx.pagerank(G)
        sorted_pr = sorted(pr.items(), key=lambda x: x[1], reverse=True)

        top_10_pr = sorted_pr[:10] 
        top_10_ids_pr = [id for id, pr in top_10_pr]

        # https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.betweenness_centrality.html
        centrality = nx.betweenness_centrality(G)
        sorted_centrality = sorted(centrality.items(), key=lambda x: x[1], reverse=True)

        top_10_centrality = sorted_centrality[:10] 
        top_10_ids_centrality = [id for id, _ in top_10_centrality]

        top_users = list(set(top_10_ids_pr + top_10_ids_centrality))

        classifier = pipeline("sentiment-analysis", model="blanchefort/rubert-base-cased-sentiment")

        user_sentiments = {}

        for post in self.data['vk'][0]['posts']:
            if post['from'] is not None and post['from']['id'] in top_users:
                text = post['text']
                result = classifier(text)[0]

                user_id = post['from']['id']
                if user_id not in user_sentiments:
                    user_sentiments[user_id] = {"POSITIVE": 0, "NEGATIVE": 0}

                if result['label'] == 'POSITIVE':
                    user_sentiments[user_id]["POSITIVE"] += 1
                elif result['label'] == 'NEGATIVE':
                    user_sentiments[user_id]["NEGATIVE"] += 1

            for reply in post['replies']:
                if reply['sender_id'] in top_users:
                    text = reply['text']
                    result = classifier(text)[0]

                    user_id = reply['sender_id']
                    if user_id not in user_sentiments:
                        user_sentiments[user_id] = {"POSITIVE": 0, "NEGATIVE": 0}

                    if result['label'] == 'POSITIVE':
                        user_sentiments[user_id]["POSITIVE"] += 1
                    elif result['label'] == 'NEGATIVE':
                        user_sentiments[user_id]["NEGATIVE"] += 1

        loyalty_scores = {}

        for user_id, sentiment_info in user_sentiments.items():
            positive = sentiment_info['POSITIVE']
            negative = sentiment_info['NEGATIVE']

            loyalty_score = positive / (positive + negative) if (positive + negative) > 0 else 0
            loyalty_scores[user_id] = loyalty_score

        if len(loyalty_scores.values()) != 0:
            return sum(loyalty_scores.values()) / len(loyalty_scores.values())
        else:
            return 0
            
    def channel_citation_index(self):
        '''
        Индекс цитируемости
        '''
        channel_citation_sum = 0
        post_count = 0

        for post in self.data['vk'][0]['posts']:
            if post['views'] is not None:
                views = post['views']
                forwards = post['forwards']
                
                channel_citation_index = forwards / views
                
                channel_citation_sum += channel_citation_index
                post_count += 1

        if post_count != 0:
            return (channel_citation_sum / post_count) / 0.05
        else:
            return 0
        
    def calculate_model(self, weight=tuple((100 / 11) for _ in range(11))):
        '''
        Итоговые значения
        weight - кортеж из весов (по умолчанию все метрики равнозначны)
        '''
        name_groups = ['Показатели вовлеченности аудитории', 'Метрики для оценивания обратной связи аудитории',
            'Метрики эффективности коммуникаций бренда', 'Метрики для оценивания взаимодействия с инфлюенсерами',
            'Метрики информационного присутствия бренда']
        
        indicators = (self.brand_follower_ratio(), self.user_engagement_ratio(), self.top_audience_ratio(), self.love_rate(),
                    self.trending_content_sentiment_ratio(), self.net_promoter_score(), self.brand_response_to_comments(), 
                    self.brand_reaction_to_negativity(), self.brand_responsiveness(), self.influencer_sentiment_ratio(), self.channel_citation_index())
        
        # при подсчете рейтинга веса получаем от клиента
        result = [w * v for w, v in zip(weight, indicators)] 
        result_model = sum(result)
        
        # в рекомендациях веса не меняем
        result_min = [w * v for w, v in zip(tuple((100 / 11) for _ in range(11)), indicators)] 
        group = [sum(result_min[:3])/3, sum(result_min[3:6])/3, sum(result_min[6:9])/3, result_min[9], result_min[10]]
        min_group = group.index(min(group))
        
        return result_model, name_groups[min_group] # result_model - итоговый рейтинг, (name_groups[min_group]) - название слабой группы 


user = Model(data)
print(f"Доля подписчиков сообщества среди активных пользователей: {user.brand_follower_ratio()}") 
print(f"Доля комментирующих среди активных пользователей: {user.user_engagement_ratio()}")
print(f"Доля подписчиков из топа активных пользователей: {user.top_audience_ratio()}") 
print(f"Средний Love rate для постов: {user.love_rate()}")
print(f"Доля позитивных комментариев из топа наиболее обсуждаемых постов: {user.trending_content_sentiment_ratio()}")
print(f"Лояльность аудитории бренда: {user.net_promoter_score()}")
print(f"Среднее время отклика компании на комментарии: {user.brand_response_to_comments()}")
print(f"Среднее время отклика бренда на негативные комментарии: {user.brand_reaction_to_negativity()}") 
print(f"Отзывчивость бренда: {user.brand_responsiveness()}")
print(f"Доля лояльных инфлюенсеров: {user.influencer_sentiment_ratio()}") 
print(f"Индекс цитируемости: {user.channel_citation_index()}")

print(f"Итоговый рейтинг: {user.calculate_model()}")

Доля подписчиков сообщества среди активных пользователей: 0.9428571428571428
Доля комментирующих среди активных пользователей: 0.1364914759727622
Доля подписчиков из топа активных пользователей: 1.0
Средний Love rate для постов: 0.29066381564192006
Доля позитивных комментариев из топа наиболее обсуждаемых постов: 0.8747222222222222
Лояльность аудитории бренда: 0.7304546087440823
Среднее время отклика компании на комментарии: 1.0
Среднее время отклика бренда на негативные комментарии: 1.0
Отзывчивость бренда: 0.8947368421052632
Доля лояльных инфлюенсеров: 0.7721836788283892
Индекс цитируемости: 0.01351995857259055
Итоговый рейтинг: (69.59663404494884, 'Метрики информационного присутствия бренда')
